In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Set the path to the dataset
dataset_folder = "train"

# Initialize data and labels lists
data = []
labels = []

# Image processing parameters
image_size = (64, 64)  # Resize images to 64x64 pixels

# Load images and assign labels based on filenames
for img_name in os.listdir(dataset_folder):
    img_path = os.path.join(dataset_folder, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
    img = cv2.resize(img, image_size)  # Resize image
    img = img.flatten()  # Flatten the image to a 1D array
    data.append(img)

   
    # Assign label based on filename
    if "cat" in img_name:
        labels.append("cat")
    elif "dog" in img_name:
        labels.append("dog")

# Convert lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Encode labels to integers (cats=0, dogs=1)
le = LabelEncoder()
labels = le.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the SVM model
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Example prediction on a new image
test_image_path = "test1"
test_img = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)
test_img = cv2.resize(test_img, image_size)
test_img = test_img.flatten().reshape(1, -1)
prediction = svm.predict(test_img)
print(f"Predicted class: {le.inverse_transform(prediction)[0]}")

